In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "2g").\
        getOrCreate()

22/02/20 01:45:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [8]:
df = spark.read.option("header", "true").csv("data/final_dataset.csv")
#df.printSchema()

In [9]:
from pyspark.sql.functions import split, explode, col, desc

#Limpando os dados
df = df.where(col("Label") != "Benign") #somente fluxos de trafego ddos
df = df.where(col("Ack Flag Cnt") == "0") #somente fluxos em forward direction(client -> servidor)

In [11]:
from pyspark.sql.functions import split, explode, col, desc
from pyspark.sql.functions import col, avg
from pyspark.sql.types import StringType, IntegerType

#Quais IPs foram mais atacados:
dst_ips = df.groupBy("Dst IP").count().orderBy(col("count").desc())
dst_ips.show()

#Quais IPs foram mais spoofados? 
src_ips = df.groupBy("Src IP").count().orderBy(col("count").desc())
src_ips.show()

#Quais portas/serviços foram mais atacados 
dst_port = df.groupBy("Dst Port").count().orderBy(col("count").desc())
dst_port.show()

#Quais IPs/serviços foram mais atacados?
dst_ip_port = df.groupBy("Dst IP","Dst Port").count().orderBy(col("count").desc())
dst_ip_port.show()


#Media de duração de um fluxo de ataque: (min)
df = df.withColumn("Flow Duration",col("Flow Duration").cast(IntegerType()))
flow = df.select(avg(col('Flow Duration'))/60000)
flow.show()


+---------------+------+
|         Dst IP| count|
+---------------+------+
|   172.31.69.25|447572|
|   172.31.69.28|165480|
| 18.216.200.189| 93097|
| 18.218.229.235| 92926|
|  18.218.115.60| 92442|
|   18.218.11.51| 92123|
|   18.216.24.42| 91854|
|     18.219.9.1| 91847|
|   18.219.32.43| 91834|
|  18.218.55.126| 91688|
|  52.14.136.135| 91269|
|    18.219.5.43| 91061|
|  97.74.144.108|  9142|
|   203.73.24.75|  8214|
|      74.55.1.4|  4348|
|  67.220.214.50|  3024|
|  69.84.133.138|  2062|
|  97.74.104.201|  1662|
|   69.192.24.88|  1569|
|208.113.162.153|   751|
+---------------+------+



+--------------+------+
|        Src IP| count|
+--------------+------+
|  172.31.69.28|920141|
|  13.59.126.31|105550|
|    18.219.9.1| 46150|
|18.218.229.235| 46035|
|18.216.200.189| 45825|
| 52.14.136.135| 45810|
| 18.218.55.126| 45685|
|   18.219.5.43| 45531|
|  18.216.24.42| 45467|
|  18.218.11.51| 45397|
|  18.219.32.43| 45256|
| 18.218.115.60| 44410|
|18.219.211.138| 26860|
| 18.219.193.20| 17704|
|  192.168.56.1| 12796|
| 18.217.165.70|  7372|
|192.168.56.102|  4990|
| 192.168.2.109|  3633|
| 192.168.2.110|  2906|
| 192.168.3.114|  1296|
+--------------+------+
only showing top 20 rows



+--------+------+
|Dst Port| count|
+--------+------+
|      80|539791|
|      21|105550|
|     443|    81|
|   58819|    60|
|   57051|    60|
|   55371|    60|
|   57904|    60|
|   56315|    60|
|   56525|    60|
|   56831|    60|
|   58163|    60|
|   57085|    60|
|   57112|    60|
|   57464|    60|
|   57483|    60|
|   57496|    60|
|   57569|    60|
|   57608|    60|
|   58185|    60|
|   58197|    60|
+--------+------+
only showing top 20 rows



+---------------+--------+------+
|         Dst IP|Dst Port| count|
+---------------+--------+------+
|   172.31.69.25|      80|342022|
|   172.31.69.28|      80|165480|
|   172.31.69.25|      21|105550|
|  97.74.144.108|      80|  9142|
|   203.73.24.75|      80|  8214|
|      74.55.1.4|      80|  4348|
|  67.220.214.50|      80|  3010|
|  69.84.133.138|      80|  2062|
|  97.74.104.201|      80|  1595|
|   69.192.24.88|      80|  1569|
|208.113.162.153|      80|   751|
|  18.218.55.126|      80|   164|
|   18.216.24.42|      80|   161|
|    18.219.5.43|      80|   160|
| 18.218.229.235|      80|   160|
|   18.219.32.43|      80|   160|
|     18.219.9.1|      80|   159|
|  18.218.115.60|      80|   159|
|   18.218.11.51|      80|   159|
| 18.216.200.189|      80|   158|
+---------------+--------+------+
only showing top 20 rows



+----------------------------+
|(avg(Flow Duration) / 60000)|
+----------------------------+
|          25.523000560509136|
+----------------------------+

